# Developer

task developer

--CASA Developer--

&nbsp;

--- In progress: merge of plotweather
---&nbsp;&nbsp;

&nbsp;

Aim of the changes:&nbsp; incorporate
plotweather functionalities into gencal so plotweather can be retired.
Plotweather calculates opacities based on the WEATHER table and/or a
seasonal model. Plotweather also produces a plot of the&nbsp;models
opacities, wind, temperature, dew point, pwv, and solar elevation. The
following is the new task description with proposed changes in red
color. Green are annotations and
dpotential discussion points.&nbsp;

&nbsp;

---

&nbsp;

&nbsp;

The&nbsp;**gencal**&nbsp;task provides a means of specifying
antenna-based calibration values manually. The values are put in
designated tables and applied to the data using other tasks
(<a href="https://casa.nrao.edu/casadocs-devel/stable/task_applycal" class="task-name">applycal</a>,&nbsp;<a href="https://casa.nrao.edu/casadocs-devel/stable/task_gaincal" class="task-name">gaincal</a>,&nbsp;<a href="https://casa.nrao.edu/casadocs-devel/stable/task_bandpass" class="task-name">bandpass</a>,
etc.). Several specialized calibrations are also generated
with&nbsp;**gencal**.

&nbsp;



## Calibration types:&nbsp;*caltype*

The&nbsp;**gencal**&nbsp;task supports many different calibration types
via the&nbsp;*caltype*&nbsp;parameter. These are listed here in two
groups. Many of these options are part of&nbsp;[Preparing for
Calibration](https://casa.nrao.edu/casadocs-devel/synthesis-calibration/preparing-for-calibration "prep for cal")&nbsp;and
more information about how they work can be found in that section.



#### Manual&nbsp;*caltype*s

The following enable directly specifying calibration factors for each
specified&nbsp;*pol*,&nbsp;*antenna*,&nbsp;*spw*. Except where noted,
each expects one real&nbsp;*parameter*&nbsp;value per
specified&nbsp;*pol*,&nbsp;*antenna*, and&nbsp;*spw*.

-   'amp'= amplitude correction
-   'ph' = phase correction
-   'sbd'= single-band delay (phase-frequency slope for each spw)
-   'mbd'= multi-band delay (phase-frequency slope over all spw)
-   'opac' =
    Tropospheric opacity&nbsp;
-   'antpos' = ITRF antenna position corrections (3 real parameters \[in
    m\] for each antenna; see below)
-   'antposvla' = VLA-centric antenna position corrections (3 real
    parameters \[in m\] for each antenna; see below)



#### Specialized&nbsp;*caltype*s

The following&nbsp;*caltype*&nbsp;options automatically generate
caltables from ancilliary information found in the MS or elsewhere.
The&nbsp;*pol, antenna, spw,&nbsp;*and&nbsp;*parameter&nbsp;*options are
ignored for these.

-   'tsys' = Tsys from the MS.SYSCAL table (ALMA)
-   'opac' = Tropospheric opacity (either
    using a combination of a atmospheric model based on the recorded
    weather data and a seasonal model). Known opacities&nbsp;can
    be&nbsp;supplied by 1 real parameter \[in nepers\] per antenna,
    spw)
-   'swpow' = VLA switched-power gains from MS.SYSPOWER, CALDEVICE
-   'rq' = VLA requantizer gains \_only\_
-   'swp/rq' = VLA switched-power gains divided by requantizer gain
-   'gc' = Gain curve (zenith-angle-dependent gain) (VLA only;
    auto-lookup)
-   'eff' = Antenna efficiency (sqrt(K/Jy)) (VLA only)
-   'gceff' = Gain curve and efficiency (VLA only)
-   'tecim' = Time-dependent TEC image specified
    in&nbsp;*infile*&nbsp;subparameter
-   'antpos' = For VLA datasets, automatic lookup of antenna position
    corrections if*&nbsp;antenna=''*

&nbsp;



## Specifying calibration values:&nbsp;*pol, antenna, spw, parameter*

Generic calibration values for the "manual&nbsp;*caltype*s" listed above
should be specified in the&nbsp;*parameter*&nbsp;argument as a list. The
length of the list must correspond to the net length of the specific
polarizations, antennas, and spws specified in
the&nbsp;*pol*,&nbsp;*antenna*, and&nbsp;*spw*&nbsp;selection
arguments.&nbsp; The values specified in&nbsp;*parameter*&nbsp;will be
duplicated over all members of any selection axis that is not explicitly
specified (*pol*='',&nbsp;*spw*='' and/or&nbsp;*antenna*='') E.g.,
if&nbsp;*pol*=*antenna*=*spw*='', it only makes sense to specify a
single&nbsp;*parameter*&nbsp;value (or three,
for&nbsp;*antpos*&nbsp;and&nbsp;*antposvla*), and this will be
duplicated for all pols, antennas, and spws. If
multiple&nbsp;*parameter*&nbsp;values are specified, at least one
of&nbsp;*pol*,&nbsp;*spw*, or&nbsp;*antenna*&nbsp;must be non-trivial,
and the number of values in&nbsp;*parameter*&nbsp;must be consistent
with the range of specified&nbsp;*pol*,&nbsp;*spw*,
and/or&nbsp;*antenna*. E.g., if only a non-trivial&nbsp;*spw*selection
is specified, then the&nbsp;*parameter*&nbsp;value list should match the
number of spws specified, and these values will be duplicated for all
polarizations and antennas. If more than one of&nbsp;*pol*,&nbsp;*spw*,
and&nbsp;*antenna*&nbsp;is non-trivially specified, the number
of&nbsp;*parameter&nbsp;*values specified should match the product of
the number specified selection elements.
The&nbsp;*parameter*&nbsp;values should be sorted
by&nbsp;*pol*&nbsp;(fastest),&nbsp;*antenna*,
and&nbsp;*spw*&nbsp;(slowest). Un-specified elements on non-trivially
specified axes will be filled with nominal values (i.e., it is not
necessary to exhaustively specify all elements on any axis or use
nominal&nbsp;*parameter*&nbsp;values explicitly).&nbsp;**Please consult
the examples for additional guidance.**&nbsp;There is currently no
support for time-dependent calibration specfication; in all cases, the
specified&nbsp;*parameter*&nbsp;values will be assumed constant in time
(though their impact on the data may be time-dependent, depending on
the&nbsp;*caltype*).

The same&nbsp;*caltable*&nbsp;can be specified for multiple runs
of&nbsp;**gencal**, in which case the
specified&nbsp;*parameter*&nbsp;values will be incorporated
cumulatively. E.g., amplitude-like values (*caltype='amp'*) multiply and
phase-like values ('ph', 'sbd','mbd','antpos') add. Also, 'amp' and 'ph'
calibrations can be incorporated into the same&nbsp;*caltable*&nbsp;(in
separate cumulative runs), but each of the other types require their own
unique&nbsp;*caltable*. A mechanism for specifying manual corrections
via a text file will be provided in the future.

The calibration tables generated by&nbsp;**gencal**&nbsp;can be applied
to the data in all other tasks that accept specified calibration for
(pre-)application,
e.g.,&nbsp;**<a href="https://casa.nrao.edu/casadocs-devel/task_applycal" class="task-name">applycal.</a>**&nbsp;<a href="https://casa.nrao.edu/casadocs-devel/stable/task_gaincal" class="task-name">gaincal</a>,&nbsp;<a href="https://casa.nrao.edu/casadocs-devel/stable/task_bandpass" class="task-name">bandpass</a>,
etc.

Consult the Examples for more information on the
many&nbsp;*caltype*&nbsp;options in&nbsp;**gencal**.



## Notes on specific&nbsp;*caltype*s

-   'antpos'&nbsp; For antenna position corrections
    (*caltype='antpos'*), the antenna position offsets are specified in
    the ITRF frame. For the Karl G. Jansky VLA, automated lookup of the
    antenna position corrections is enabled when antenna is unspecified
    (*antenna=''*) for this&nbsp;*caltype*. Note that this requires
    internet connection to access the VLA antenna position correction
    site.
-   'antposvla'&nbsp; For (old) pre-upgrade VLA position corrections,
    specify the values in the VLA-centric frame
    and&nbsp;**gencal**&nbsp;will rotate them to ITRF before storing
    them in the output caltable.
-   VLA switched power calibration is supported in three modes: 'swpow'
    (formerly 'evlagain', a syntax which has been deprecated) yields the
    formal VLA switched power calibration which describes voltage gain
    as sqrt(Pdif/Tcal) (used to correct the visibility data) and Tsys as
    Psum\*Tcal/Pdif/2 (used to correct the weights). 'swpow' implicitly
    includes any requantizer gain scale and adjustments. 'rq' yields
    only the requantizer voltage gains (Tsys is set to 1.0 to avoid
    weight adjustments). 'swp/rq' yields the ordinary switched power
    voltage gains divided by the requantizer voltage gain (Tsys is
    calculated normally). The 'rq' and 'swp/rq' modes are are mainly
    intended for testing and evaluating the VLA switched power systems.
-   &nbsp;For&nbsp;*caltype='opac'*,
    only constant (in time) opacities are supported
    via&nbsp;**gencal**.&nbsp;&nbsp;
-   *caltype='opac'* will open a sub-menu
    with parameters *seasonal_weight*
    and*&nbsp;plotweather*.&nbsp;Default for *seasonal_weight* is -1. In
    that case, the '*parameters*'&nbsp;&nbsp;need to be specified with a
    list of opacity values for each spw. When *seasonal_model* is
    between 0 and 1, the opacity&nbsp;calibration table
    is&nbsp;calculated based on opacities&nbsp;using the weighted
    average of a seasonal model and a model based on the WEATHER table
    of the MS. The computed opacity values are listed in the logger
    (and can be returned as a
    dictionary?). The parameter *plotweather* generates a plot
    with weather parameters solar elevation, wind speed and direction,
    temperature, dew point, pwv, as a function of time (pwv shows three
    graphs, one for the sesonal model, one using the weather data, and
    the combination defined by the seasonal_model weights). The plot
    also contains the opacities of the three different models. When
    plotweather is an empty string, no plot is generated. NOTE: this will allow backward compatibility
    for the opacities, but not for the plotting behavior of plotweather,
    it is not allowed anymore to generate a plot that is automatically
    named. Only opacities that are
    constant in time are currently supported.&nbsp;
-   &nbsp;For gaincurve and efficiency (*caltype='gc'*,&nbsp;*'gceff'*,
    or&nbsp;*'eff'*), observatory-provided factors are determined per
    spw according to the observing frequencies. The parameter argument
    is ignored. These&nbsp;*caltype*s are currently only supported for
    VLA (including pre-upgrade VLA) processing. (Appropriate factors for
    ALMA are TBD.)